In [19]:
import numpy as np
import pandas as pd
import arviz as az
from collections import defaultdict

def create_manual_trace(att_str, def_str, baseline, home_adv, n_samples=1000, pin_first_team=True):
    """
    Create a manual trace dictionary from parameter arrays.
    
    Parameters:
    -----------
    att_str : array-like, shape (n_teams,)
        Attack strength for each team
    def_str : array-like, shape (n_teams,)
        Defense strength for each team
    baseline : float
        Baseline goal rate
    home_adv : float
        Home advantage parameter
    n_samples : int
        Number of posterior samples to generate (default: 1000)
    pin_first_team : bool
        If True, keeps index 0 team values fixed and adjusts other 19 teams to sum to zero
        
    Returns:
    --------
    dict : Manual trace dictionary that can be used with predict_match
    """
    att_str = np.array(att_str, dtype=float).copy()
    def_str = np.array(def_str, dtype=float).copy()
    
    # Pin first team and adjust others to maintain zero-sum
    if pin_first_team:
        # Calculate what the sum of teams 1-19 needs to be to make total = 0
        # If Arsenal = 0.5 and we want total = 0, then others must sum to -0.5
        required_att_sum = -att_str[0]
        required_def_sum = -def_str[0]
        
        # Current sum of teams 1-19
        current_att_sum = att_str[1:].sum()
        current_def_sum = def_str[1:].sum()
        
        # Calculate adjustment needed for each of the 19 teams
        # We distribute the difference equally across all 19 teams
        att_adjustment = (required_att_sum - current_att_sum) / 19
        def_adjustment = (required_def_sum - current_def_sum) / 19
        
        # Apply adjustment to teams 1-19
        att_str[1:] += att_adjustment
        def_str[1:] += def_adjustment
    
    # Replicate the values n_samples times to create posterior samples
    return {
        'att_str': np.tile(att_str, (n_samples, 1)),  # Shape: (n_samples, n_teams)
        'def_str': np.tile(def_str, (n_samples, 1)),  # Shape: (n_samples, n_teams)
        'baseline': np.full(n_samples, baseline),      # Shape: (n_samples,)
        'home_adv': np.full(n_samples, home_adv)       # Shape: (n_samples,)
    }

def get_team_strengths(trace, team_mapping, is_manual=False):
    """
    Extract and display team strengths from a trace.
    
    Parameters:
    -----------
    trace : dict or arviz.InferenceData
        The trace to extract from
    team_mapping : dict
        Mapping of team names to indices
    is_manual : bool
        Whether the trace is in manual format
        
    Returns:
    --------
    pd.DataFrame : DataFrame with team strengths
    """
    if is_manual:
        att_str = trace['att_str'][0]
        def_str = trace['def_str'][0]
        baseline = trace['baseline'][0]
        home_adv = trace['home_adv'][0]
    else:
        n_teams = len(team_mapping)
        att_str = np.mean(trace.posterior['att_str'].values.reshape(-1, n_teams), axis=0)
        def_str = np.mean(trace.posterior['def_str'].values.reshape(-1, n_teams), axis=0)
        baseline = np.mean(trace.posterior['baseline'].values.flatten())
        home_adv = np.mean(trace.posterior['home_adv'].values.flatten())
    
    # Create DataFrame
    team_names = [name for name, idx in sorted(team_mapping.items(), key=lambda x: x[1])]
    df = pd.DataFrame({
        'team': team_names,
        'attack': att_str,
        'defense': def_str
    })
    
    df = df.sort_values('attack', ascending=False)
    
    print(f"\nBaseline: {baseline:.4f}")
    print(f"Home Advantage: {home_adv:.4f}")
    print(f"\nAttack sum: {att_str.sum():.6f}")
    print(f"Defense sum: {def_str.sum():.6f}")
    
    return df

def predict_match(home_team, away_team, trace, team_mapping, is_manual=False):
    """
    Predict goals for a match between two teams using loaded trace or manual trace
    
    Parameters:
    -----------
    home_team : str
        Name of home team
    away_team : str
        Name of away team
    trace : arviz.InferenceData or dict
        Either a loaded ArviZ trace or a manual trace dictionary
    team_mapping : dict
        Mapping of team names to indices
    is_manual : bool
        If True, treat trace as a manual dictionary. If False, treat as ArviZ object
    """
    
    # Get team indices
    home_idx = team_mapping[home_team]
    away_idx = team_mapping[away_team]
    
    # Extract posterior samples based on trace type
    if is_manual:
        # Manual trace - already in correct format
        att_str = trace['att_str']
        def_str = trace['def_str']
        baseline = trace['baseline']
        home_adv = trace['home_adv']
    else:
        # ArviZ trace - extract from posterior
        att_str = trace.posterior['att_str'].values.reshape(-1, len(team_mapping))
        def_str = trace.posterior['def_str'].values.reshape(-1, len(team_mapping))
        baseline = trace.posterior['baseline'].values.flatten()
        home_adv = trace.posterior['home_adv'].values.flatten()
    
    # Calculate expected goals
    home_goals_lambda = np.exp(
        baseline + 
        att_str[:, home_idx] + 
        def_str[:, away_idx] + 
        home_adv
    )
    
    away_goals_lambda = np.exp(
        baseline + 
        att_str[:, away_idx] + 
        def_str[:, home_idx]
    )
    
    home_goals_pred = np.random.poisson(home_goals_lambda)
    away_goals_pred = np.random.poisson(away_goals_lambda)
    
    return {
        'home_team': home_team,
        'away_team': away_team,
        'home_goals_expected': np.mean(home_goals_lambda),
        'away_goals_expected': np.mean(away_goals_lambda),
        'home_goals_median': np.median(home_goals_pred),
        'away_goals_median': np.median(away_goals_pred),
        'home_win_prob': np.mean(home_goals_pred > away_goals_pred),
        'draw_prob': np.mean(home_goals_pred == away_goals_pred), 
        'away_win_prob': np.mean(home_goals_pred < away_goals_pred)
    }

def simulate_full_season(trace, team_mapping, is_manual=False):
    """
    Simulate a full season where every team plays every other team home and away
    
    Parameters:
    -----------
    trace : arviz.InferenceData or dict
        Either a loaded ArviZ trace or a manual trace dictionary
    team_mapping : dict
        Mapping of team names to indices
    is_manual : bool
        If True, treat trace as a manual dictionary
    """
    
    teams = list(team_mapping.keys())
    
    # Initialize league table with xG stats
    league_table = {team: {
        'played': 0,
        'wins': 0,
        'draws': 0,
        'losses': 0,
        'goals_for': 0,
        'goals_against': 0,
        'goal_difference': 0,
        'xg_for': 0.0,
        'xg_against': 0.0,
        'xg_difference': 0.0,
        'points': 0
    } for team in teams}
    
    all_matches = []
    
    # Generate all possible matches
    for home_team in teams:
        for away_team in teams:
            if home_team != away_team:
                # Predict the match
                result = predict_match(home_team, away_team, trace, team_mapping, is_manual)
                
                # Use Poisson goals
                home_goals = np.random.poisson(result['home_goals_expected'])
                away_goals = np.random.poisson(result['away_goals_expected'])
                
                # Store match result
                match_result = {
                    'home_team': home_team,
                    'away_team': away_team,
                    'home_goals': home_goals,
                    'away_goals': away_goals,
                    'home_expected': result['home_goals_expected'],
                    'away_expected': result['away_goals_expected']
                }
                all_matches.append(match_result)
                
                # Update league table for home team
                league_table[home_team]['played'] += 1
                league_table[home_team]['goals_for'] += home_goals
                league_table[home_team]['goals_against'] += away_goals
                league_table[home_team]['xg_for'] += result['home_goals_expected']
                league_table[home_team]['xg_against'] += result['away_goals_expected']
                
                if home_goals > away_goals:
                    league_table[home_team]['wins'] += 1
                    league_table[home_team]['points'] += 3
                elif home_goals == away_goals:
                    league_table[home_team]['draws'] += 1
                    league_table[home_team]['points'] += 1
                else:
                    league_table[home_team]['losses'] += 1
                
                # Update league table for away team
                league_table[away_team]['played'] += 1
                league_table[away_team]['goals_for'] += away_goals
                league_table[away_team]['goals_against'] += home_goals
                league_table[away_team]['xg_for'] += result['away_goals_expected']
                league_table[away_team]['xg_against'] += result['home_goals_expected']
                
                if away_goals > home_goals:
                    league_table[away_team]['wins'] += 1
                    league_table[away_team]['points'] += 3
                elif away_goals == home_goals:
                    league_table[away_team]['draws'] += 1
                    league_table[away_team]['points'] += 1
                else:
                    league_table[away_team]['losses'] += 1
    
    # Calculate differences
    for team in teams:
        league_table[team]['goal_difference'] = (
            league_table[team]['goals_for'] - league_table[team]['goals_against']
        )
        league_table[team]['xg_difference'] = (
            league_table[team]['xg_for'] - league_table[team]['xg_against']
        )
    
    return league_table, all_matches

def print_league_table(league_table):
    """Print formatted league table with xG stats"""
    
    df = pd.DataFrame.from_dict(league_table, orient='index')
    df = df.sort_values(['points', 'goal_difference', 'goals_for'], ascending=[False, False, False])
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'team'}, inplace=True)
    df.index = df.index + 1
    
    print("\n" + "="*110)
    print("PREDICTED FINAL LEAGUE TABLE (with Expected Goals)")
    print("="*110)
    print(f"{'Pos':>3} {'Team':15} {'P':>2} {'W':>2} {'D':>2} {'L':>2} {'GF':>3} {'GA':>3} {'GD':>4} {'xGF':>5} {'xGA':>5} {'xGD':>5} {'Pts':>3}")
    print("-"*110)
    
    for pos, row in df.iterrows():
        print(f"{pos:>3} {row['team']:15} {row['played']:>2} {row['wins']:>2} "
              f"{row['draws']:>2} {row['losses']:>2} {row['goals_for']:>3} "
              f"{row['goals_against']:>3} {row['goal_difference']:>+4} "
              f"{row['xg_for']:>5.1f} {row['xg_against']:>5.1f} {row['xg_difference']:>+5.1f} {row['points']:>3}")
    
    return df

def run_multiple_seasons(n_simulations, trace, team_mapping, is_manual=False):
    """
    Run multiple season simulations and return averaged results
    
    Parameters:
    -----------
    n_simulations : int
        Number of seasons to simulate
    trace : arviz.InferenceData or dict
        Either a loaded ArviZ trace or a manual trace dictionary
    team_mapping : dict
        Mapping of team names to indices
    is_manual : bool
        If True, treat trace as a manual dictionary
    """
    
    teams = list(team_mapping.keys())
    
    # Initialize accumulated statistics
    accumulated_stats = {team: {
        'total_points': 0,
        'total_wins': 0,
        'total_draws': 0, 
        'total_losses': 0,
        'total_goals_for': 0,
        'total_goals_against': 0,
        'total_xg_for': 0.0,
        'total_xg_against': 0.0,
        'position_sum': 0
    } for team in teams}
    
    # Track position frequencies
    position_counts = {team: [0] * 20 for team in teams}
    
    print(f"Running {n_simulations} season simulations...")
    
    for sim in range(n_simulations):
        if (sim + 1) % 1000 == 0:
            print(f"Completed {sim + 1} simulations...")
        
        # Run single season simulation
        league_table, _ = simulate_full_season(trace, team_mapping, is_manual)
        
        # Convert to DataFrame and sort
        df = pd.DataFrame.from_dict(league_table, orient='index')
        df = df.sort_values(['points', 'goal_difference', 'goals_for'], 
                           ascending=[False, False, False])
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'team'}, inplace=True)
        
        # Accumulate statistics
        for pos, row in df.iterrows():
            team = row['team']
            final_position = pos + 1
            
            accumulated_stats[team]['total_points'] += row['points']
            accumulated_stats[team]['total_wins'] += row['wins']
            accumulated_stats[team]['total_draws'] += row['draws']
            accumulated_stats[team]['total_losses'] += row['losses']
            accumulated_stats[team]['total_goals_for'] += row['goals_for']
            accumulated_stats[team]['total_goals_against'] += row['goals_against']
            accumulated_stats[team]['total_xg_for'] += row['xg_for']
            accumulated_stats[team]['total_xg_against'] += row['xg_against']
            accumulated_stats[team]['position_sum'] += final_position
            
            position_counts[team][pos] += 1
    
    # Calculate averages
    avg_results = []
    for team in teams:
        stats = accumulated_stats[team]
        avg_results.append({
            'team': team,
            'avg_points': stats['total_points'] / n_simulations,
            'avg_wins': stats['total_wins'] / n_simulations,
            'avg_draws': stats['total_draws'] / n_simulations,
            'avg_losses': stats['total_losses'] / n_simulations,
            'avg_goals_for': stats['total_goals_for'] / n_simulations,
            'avg_goals_against': stats['total_goals_against'] / n_simulations,
            'avg_xg_for': (stats['total_xg_for'] / n_simulations)/ 38,
            'avg_xg_against':( stats['total_xg_against'] / n_simulations) /38,
            'avg_position': stats['position_sum'] / n_simulations
        })
    
    # Create DataFrame and sort by average points
    avg_df = pd.DataFrame(avg_results)
    avg_df['avg_goal_difference'] = avg_df['avg_goals_for'] - avg_df['avg_goals_against']
    avg_df['avg_xg_difference'] = avg_df['avg_xg_for'] - avg_df['avg_xg_against']
    avg_df = avg_df.sort_values(['avg_points', 'avg_goal_difference', 'avg_goals_for'], 
                               ascending=[False, False, False])
    avg_df.reset_index(drop=True, inplace=True)
    avg_df.index = avg_df.index + 1
    
    return avg_df, position_counts

In [20]:
team_mapping = {
    'Arsenal': 0,
    'Aston Villa': 1,
    'Bournemouth': 2,
    'Brentford': 3,
    'Brighton': 4,
    'Chelsea': 5,
    'Crystal Palace': 6,
    'Everton': 7,
    'Fulham': 8,
    'Luton': 9,      # Promoted - using Luton's values as proxy
    'Leicester': 10,  # Promoted - using Burnley's values as proxy
    'Liverpool': 11,
    'Man City': 12,
    'Man United': 13,
    'Newcastle': 14,
    'Nottm Forest': 15,
    'Sheffield United': 16,  # Promoted - using Sheff Utd's values as proxy
    'Tottenham': 17,
    'West Ham': 18,
    'Wolves': 19
}

In [23]:
def calculate_required_strengths(target_xgf, target_xga, baseline, home_adv,
                                 other_teams_att, other_teams_def):
    """
    Calculate the attack and defense strengths needed to achieve target xG.
    
    Parameters:
    -----------
    target_xgf : float
        Target expected goals FOR per game
    target_xga : float
        Target expected goals AGAINST per game
    baseline : float
        Model baseline
    home_adv : float
        Home advantage
    other_teams_att : array-like
        Attack strengths of the other 19 teams
    other_teams_def : array-like
        Defense strengths of the other 19 teams
    
    Returns:
    --------
    tuple : (required_att_strength, required_def_strength)
    """
    other_teams_att = np.array(other_teams_att)
    other_teams_def = np.array(other_teams_def)
    
    # Calculate average opponent strengths (what this team actually faces)
    avg_opp_att = np.mean(other_teams_att)
    avg_opp_def = np.mean(other_teams_def)
    
    # For xG FOR:
    # Home: exp(baseline + att + opp_def + home_adv) for each opponent
    # Away: exp(baseline + att + opp_def) for each opponent
    # We need: (sum of home_xg + sum of away_xg) / 38 = target_xgf
    
    # Average over opponents:
    # target_xgf = (exp(baseline + att + avg_opp_def + home_adv) + exp(baseline + att + avg_opp_def)) / 2
    # 2*target_xgf = exp(baseline + att + avg_opp_def) * (exp(home_adv) + 1)
    # exp(baseline + att + avg_opp_def) = 2*target_xgf / (exp(home_adv) + 1)
    # att = ln(2*target_xgf / (exp(home_adv) + 1)) - baseline - avg_opp_def
    
    required_att = np.log(2 * target_xgf / (np.exp(home_adv) + 1)) - baseline - avg_opp_def
    
    # For xG AGAINST:
    # Home: exp(baseline + opp_att + def) for each opponent
    # Away: exp(baseline + opp_att + def + home_adv) for each opponent (they get home adv)
    # Average over opponents:
    # target_xga = (exp(baseline + avg_opp_att + def) + exp(baseline + avg_opp_att + def + home_adv)) / 2
    # 2*target_xga = exp(baseline + avg_opp_att + def) * (1 + exp(home_adv))
    # exp(baseline + avg_opp_att + def) = 2*target_xga / (1 + exp(home_adv))
    # def = ln(2*target_xga / (1 + exp(home_adv))) - baseline - avg_opp_att
    
    required_def = np.log(2 * target_xga / (1 + np.exp(home_adv))) - baseline - avg_opp_att
    
    return required_att, required_def


# Usage:
baseline = 0.283
home_adv = 0.253

# First create initial trace with all teams
att_str_initial = np.array([
    0.35,   # Arsenal (placeholder, will be recalculated)
    0.162,  # Aston Villa
    -0.088, # Bournemouth
    -0.035, # Brentford
    -0.119, # Brighton
    0.26,   # Chelsea
    -0.109, # Crystal Palace
    -0.175, # Everton
    -0.089, # Fulham
    -0.181, # Luton
    -0.392, # Leicester
    0.424,  # Liverpool
    0.365,  # Man City
    -0.079, # Man United
    0.302,  # Newcastle
    -0.188, # Nottm Forest
    -0.414, # Sheffield United
    0.197,  # Tottenham
    -0.043, # West Ham
    -0.151  # Wolves
])

def_str_initial = np.array([
    -0.712, # Arsenal (placeholder, will be recalculated)
    0.086,  # Aston Villa
    0.015,  # Bournemouth
    -0.04,  # Brentford
    -0.018, # Brighton
    0.043,  # Chelsea
    -0.099, # Crystal Palace
    -0.085, # Everton
    0.021,  # Fulham
    0.314,  # Luton
    0.207,  # Leicester
    -0.217, # Liverpool
    -0.436, # Man City
    0.104,  # Man United
    0.069,  # Newcastle
    -0.04,  # Nottm Forest
    0.392,  # Sheffield United
    0.103,  # Tottenham
    0.225,  # West Ham
    0.068   # Wolves
])

# Calculate required strengths for Arsenal to achieve 1.5 xGF, 0.5 xGA
# Note: we need to account for what the other teams will be AFTER adjustment
# This is iterative - let's do a simple approach first

# Step 1: Calculate what Arsenal needs assuming others stay as is
other_att = att_str_initial[1:]
other_def = def_str_initial[1:]

att_strength, def_strength = calculate_required_strengths(
    target_xgf=2.5,
    target_xga=1.5,
    baseline=baseline,
    home_adv=home_adv,
    other_teams_att=other_att,
    other_teams_def=other_def
)

print(f"Required attack strength: {att_strength:.4f}")
print(f"Required defense strength: {def_strength:.4f}")

# Step 2: Now we need to account for the adjustment that will happen
# When we pin Arsenal, the other 19 teams get adjusted
# Let's iterate to find the true values

att_str = att_str_initial.copy()
def_str = def_str_initial.copy()

for iteration in range(10):  # Iterate until convergence
    # Set Arsenal's strengths
    att_str[0] = att_strength
    def_str[0] = def_strength
    
    # Calculate what adjustment will be made to other teams
    required_att_sum = -att_str[0]
    required_def_sum = -def_str[0]
    
    current_att_sum = att_str[1:].sum()
    current_def_sum = def_str[1:].sum()
    
    att_adjustment = (required_att_sum - current_att_sum) / 19
    def_adjustment = (required_def_sum - current_def_sum) / 19
    
    # Apply adjustment
    att_str[1:] += att_adjustment
    def_str[1:] += def_adjustment
    
    # Recalculate required strengths based on adjusted opponents
    att_strength_new, def_strength_new = calculate_required_strengths(
        target_xgf=2.5,
        target_xga=1.5,
        baseline=baseline,
        home_adv=home_adv,
        other_teams_att=att_str[1:],
        other_teams_def=def_str[1:]
    )
    
    # Check convergence
    if abs(att_strength_new - att_strength) < 0.0001 and abs(def_strength_new - def_strength) < 0.0001:
        print(f"Converged after {iteration + 1} iterations")
        break
    
    att_strength = att_strength_new
    def_strength = def_strength_new

print(f"\nFinal attack strength: {att_strength:.4f}")
print(f"Final defense strength: {def_strength:.4f}")

# Now use these values
att_str[0] = att_strength
def_str[0] = def_strength

# Create manual trace with pin_first_team=True
manual_trace = create_manual_trace(
    att_str=att_str,
    def_str=def_str,
    baseline=baseline,
    home_adv=home_adv,
    n_samples=1000,
    pin_first_team=True
)

print("\n" + "="*50)
print("Verification:")
df = get_team_strengths(manual_trace, team_mapping, is_manual=True)
print(df.head())

Required attack strength: 0.4613
Required defense strength: 0.0066
Converged after 4 iterations

Final attack strength: 0.4996
Final defense strength: 0.0143

Verification:

Baseline: 0.2830
Home Advantage: 0.2530

Attack sum: -0.000000
Defense sum: -0.000000
         team    attack   defense
0     Arsenal  0.499561  0.014272
11  Liverpool  0.416286 -0.255225
12   Man City  0.357286 -0.474225
14  Newcastle  0.294286  0.030775
5     Chelsea  0.252286  0.004775


In [24]:
# Run simulation with manual trace
league_table, matches = simulate_full_season(manual_trace, team_mapping, is_manual=True)
print_league_table(league_table)

# Run multiple seasons with manual trace
avg_results, pos_counts = run_multiple_seasons(1000, manual_trace, team_mapping, is_manual=True)
print(avg_results)


PREDICTED FINAL LEAGUE TABLE (with Expected Goals)
Pos Team             P  W  D  L  GF  GA   GD   xGF   xGA   xGD Pts
--------------------------------------------------------------------------------------------------------------
  1 Liverpool       38 27  5  6  97  41  +56  89.9  45.1 +44.8  86
  2 Arsenal         38 25  6  7 103  54  +49  96.5  58.6 +37.9  81
  3 Man City        38 22  7  9  77  42  +35  85.4  36.4 +49.1  73
  4 Crystal Palace  38 17 13  8  63  45  +18  52.5  52.4  +0.1  64
  5 Newcastle       38 18 10 10  66  49  +17  78.5  60.5 +18.0  64
  6 Aston Villa     38 16 13  9  69  61   +8  68.2  62.1  +6.1  61
  7 Tottenham       38 15 12 11  76  61  +15  70.6  63.1  +7.5  57
  8 Wolves          38 16  8 14  56  60   -4  49.9  62.0 -12.1  56
  9 Chelsea         38 16  5 17  83  74   +9  75.4  59.1 +16.3  53
 10 Brentford       38 13 10 15  57  57   +0  56.4  55.4  +1.0  49
 11 West Ham        38 14  7 17  69  73   -4  55.1  72.2 -17.1  49
 12 Bournemouth     38 14  7 17  